In [1]:
import pandas as pd
import numpy as np
import string
import re
import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from simplemma import text_lemmatizer
from nltk import sent_tokenize
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv(r"C:\Users\hp\nlp_project_dataset.csv")

In [3]:
stopWords = set(stopwords.words('turkish'))


In [4]:

def clean_text(text):
    
    text = text.replace("Haber ÖzetiTam Sürüm", "")
    text = text.replace("[/inlinetweet]", "")
    text = text.lower()
    text = re.sub("[0-9]+", "", text)
    text = re.sub("’|“|”|‘|–|—", " ", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = text_lemmatizer(text, lang="tr")
    text = " ".join([word for word in text if word not in stopWords])
    
    turkish_punctuation = ".,;:!?()[]{}'\"<>-_+=*&^%$#@~`|\\/"
    translate_table = str.maketrans("", "", turkish_punctuation)
    text = text.translate(translate_table)
    
    return text

In [5]:
def cumle_puani(vektor):
    non_zero_elems = vektor[vektor != 0]
    return non_zero_elems.mean()

In [6]:
def metin_ozetle(df_column, j, cumle_sayisi=2):
    ozet = ""
    metin = df_column.iloc[j]
    cumleler = sent_tokenize(metin)
    cumleler_temiz = [clean_text(c) for c in cumleler]
    
    vectorizer = TfidfVectorizer(norm="l1")
    X = vectorizer.fit_transform(cumleler_temiz)
    skorlar = np.array([cumle_puani(v) for v in X])
    siralama_indeksleri = np.argsort(-skorlar)
    
    print("Headline:")
    print(df["Headline"][j])
    print()
    print("Özet:")
    for i in siralama_indeksleri[:cumle_sayisi]:
        ozet += cumleler[i]
    return ozet

In [7]:
metin_ozetle(df["Haber_Metni"],18, 3)

Headline:
Kötü amaçlı mobil yazılım şimdiden dünya çapında 25 milyon cihaza bulaşmış durumda.

Özet:


'Bu cihazlardan 15 milyonu Hindistan’da.Şimdilik sadece kullanıcıların deneyimini olumsuz etkileyen bu virüs aslında çok daha tehlikeli amaçlarla kullanılabilir.Kötü amaçlı yazılım aynı zamanda Birleşik Krallık, Avustralya ve Amerika’da da yayılmış durumda.'

In [8]:
def index_ozet(j,cumle_sayisi):
    return metin_ozetle(df["Haber_Metni"],j,cumle_sayisi)
    

In [13]:
j = int(input("Haber index'i giriniz ---> "))
cumle_sayisi = int(input("Kac cumlede ozetlesin ? ---> "))

index_ozet(j,cumle_sayisi)

Haber index'i giriniz ---> 11
Kac cumlede ozetlesin ? ---> 3
Headline:
Yeni bir araştırma, petrol ve doğal gaz kuyularına yakın yaşayan annelerin bebeklerinde kalp rahatsızlığı riskinin daha yüksek olduğunu gösteriyor.

Özet:


'Ülke çapında, 17 milyon kişi bu kuyulara yakın noktalarda yaşıyor.Kalp rahatsızlıkları bebeklerde büyümenin yavaşlamasına ve beyin rahatsızlıklarına sebep olabiliyor.Doğuştan gelen kalp rahatsızlıkları, doğuştan gelen hastalıklar sebebiyle yaşamını yitiren bebekler arasında en yüksek ölüm oranına sahip.'